# Importing the Disturbance Data Into the Coalesced Data Set

In [1]:
from statsmodels.regression.linear_model import OLS
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
import pandas as pd
pd.set_option("display.max_columns", 500)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
conn = sqlite3.connect("eia")
cursor = conn.cursor()
df = pd.DataFrame(cursor.execute("""select * from combined4""").fetchall())
df.columns = [description[0] for description in cursor.description]
df = df.drop(columns=["index"])
df.head()

,average_ash_content,average_heat_content,average_mercury_content,average_sulfur_content,chlorine_content,coalmine_county,coalmine_msha_id,coalmine_name,coalmine_state,coalmine_type,contract_expiration_date,contract_type,energy_source,fuel_cost,fuel_group,moisture_content,month,natural_gas_delivery_contract_type,natural_gas_supply_contract_type,natural_gas_transportation_service,operator_id,operator_name,plant_id,plant_name,plant_state,primary_transportation_mode,purchase_type,quantity,regulated,reporting_frequency,secondary_transportation_mode,supplier,year,plant_id_gen,combined_heat_and_power_plant,nuclear_unit_id,plant_name_gen,operator_name_gen,operator_id_gen,state_gen,census_region,nerc_region,reserved_,naics_code,eia_sector_number,sector_name,reported_prime_mover,reported_fuel_type_code,aer_fuel_type_code,reserved_1,reserved_2,physical_unit_label,total_fuel_consumption_quantity,electric_fuel_consumption_quantity,total_fuel_consumption_mmbtu,elec_fuel_consumption_mmbtu,net_generation_megawatthours,year_gen,quantity_gen,elec_quantity_gen,mmbtu_per_unit_gen,tot_mmbtu_gen,elec_mmbtu_gen,netgen_gen,year_month
0,5.4,23.100,NaN,0.49,NaN,IMP,0,MINA PRIBBENOW,CL,SU,408.0,C,BIT,213.5,Coal,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,RV,N/A,259412,REG,M,N/A,INTEROCEAN COAL,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,ST,BIT,COL,None,None,short tons,4208251,4208251,94561606,94561606,9472859,2008,340778,340778,22.59,7697153,7697153,778699,2008-01
1,5.7,22.800,NaN,0.48,NaN,IMP,0,MINA PRIBBENOW,CL,SU,408.0,C,BIT,211.5,Coal,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,RV,N/A,52241,REG,M,N/A,INTEROCEAN COAL,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,ST,BIT,COL,None,None,short tons,4208251,4208251,94561606,94561606,9472859,2008,340778,340778,22.59,7697153,7697153,778699,2008-01
2,0.0,1.039,NaN,0.00,NaN,None,0,None,None,None,None,C,NG,863.1,Natural Gas,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,CA,NG,NG,None,None,mcf,3433,3433,3559,3559,1609033,2008,2400,2400,1.04,2489,2489,146327,2008-01
3,0.0,1.039,NaN,0.00,NaN,None,0,None,None,None,None,C,NG,863.1,Natural Gas,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,CT,NG,NG,None,None,mcf,30932077,30932077,32090118,32090118,2858195,2008,2781102,2781102,1.04,2884003,2884003,244784,2008-01
4,0.0,1.039,NaN,0.00,NaN,None,0,None,None,None,None,C,NG,863.1,Natural Gas,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,ST,NG,NG,None,None,mcf,837652,837652,871270,871270,87265,2008,109281,109281,1.04,113543,113543,11487,2008-01


In [3]:
df_ng = df[df["fuel_group"] == "Natural Gas"]

In [3]:
df_d = pd.read_csv("disturbances.csv")
df_d = df_d.drop(columns="Unnamed: 0")

In [22]:
df_d.Date = pd.to_datetime(df_d.Date)


In [ ]:
df_d.Date.astype()

In [24]:
df_d.Year = df_d.Date.apply(lambda x: x.year)
df_d.Month = df_d.Date.apply(lambda x: x.month)

In [28]:
df_d.head()

,Area Affected,Date,Loss (megawatts),Month,NERC Region,Number of Customers Affected,Restoration Date and Time,Time,Type of Disturbance,Utility/Power Pool,Year,Year2,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,year_month
0,Northern California,2008-01-04,500,1,WECC,2606931,5:00 p.m. January 14,4:00 a.m.,Winter Storm,Pacific Gas and Electric Company,2008,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-1
1,Sacramento County,2008-01-04,300,1,WECC,150000,4:30 p.m. January 04,7:47 a.m.,Severe Storm,Sacramento Municipal Utility District,2008,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-1
2,"San Francisco Bay Area, California",2008-01-29,NaN,1,WECC,-,12:17 p.m. January 29,5:00 a.m.,Exciter Faulted,Crockett Cogeneration,2008,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-1
3,"Arkansas, Mississippi, North Louisiana",2008-01-29,NaN,1,SERC,110000,8:00 a.m February 03,4:00 p.m.,Severe Thunderstorms,Entergy Corporation,2008,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-1
4,Southeastern Michigan,2008-01-29,NaN,1,RFC,86915,6:30 p.m. February 01,10:00 p.m.,Wind/Ice Storm,DTE Energy - Detroit Edison,2008,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008-1


In [31]:
cols = df_d.columns
cols = [col for col in cols if "Unnamed" not in col]

In [32]:
df_d = df_d[cols]

In [50]:
import numpy as np
import math

In [53]:
def replace_dash(x):
    if x == "-" or x=="Unknown":
        return 0
    else:
        return x
    
df_d[ 'Number of Customers Affected'] = df_d[ 'Number of Customers Affected'].apply(replace_dash)

In [56]:
def fixna(x):
    if x.temp == True:
        return 0
    else:
        return x["Number of Customers Affected"]
    
df_d[ 'Number of Customers Affected'] = df_d.apply(fixna, axis=1)

In [52]:
df_d["temp"] = df_d[ 'Number of Customers Affected'].isna()

In [58]:
df_d[ 'Number of Customers Affected'] = df_d[ 'Number of Customers Affected'].astype(int)

In [26]:

df_d["year_month"] = df_d.apply(lambda x: str(x.Year) + "-" + str(x.Month), axis=1)

In [67]:
df_d.to_csv("disturbances.csv")

In [68]:
df_d = pd.read_csv("disturbances.csv")
df_d = df_d.drop(columns="Unnamed: 0")

In [5]:
df_d["NERC Region"].value_counts()

RFC                                 321
SERC                                217
WECC                                216
NPCC                                 86
TRE                                  81
SPP                                  56
MRO                                  29
PR                                   29
FRCC                                 22
REC                                   1
ERCOT                                 1
HICC                                  1
Southwest Public Service Company      1
TE                                    1
HI                                    1
NP                                    1
AES Greenidge and Cayuga0RFC          1
Name: NERC Region, dtype: int64

In [6]:
def make_nerc(x):
    return str(x.year_month) + "-" + str(x["NERC Region"])

df_d["year_month_nerc"] = df_d.apply(make_nerc, axis=1)

In [7]:
df_affect = df_d.groupby("year_month_nerc").sum().reset_index()[["year_month_nerc", "Number of Customers Affected"]]

In [8]:
df_ct = df_d.groupby("year_month_nerc").count().reset_index()[["year_month_nerc", "Number of Customers Affected"]]

In [9]:
df_nercs = df_affect.merge(df_ct, on="year_month_nerc")

In [10]:
df_nercs["year"] = df_nercs["year_month_nerc"].apply(lambda x: x[0:4])

In [11]:
df_nercs["month"] = df_nercs["year_month_nerc"].apply(lambda x: x.split("-")[1])

In [12]:
df_nercs["nerc"] = df_nercs["year_month_nerc"].apply(lambda x: x.split("-")[2])

In [13]:
df_nercs

,year_month_nerc,Number of Customers Affected_x,Number of Customers Affected_y,year,month,nerc
0,2008-1-NPCC,54316,1,2008,1,NPCC
1,2008-1-RFC,131915,2,2008,1,RFC
2,2008-1-SERC,110000,1,2008,1,SERC
3,2008-1-WECC,2756931,3,2008,1,WECC
4,2008-10-NPCC,52000,1,2008,10,NPCC
5,2008-10-SERC,0,1,2008,10,SERC
6,2008-11-PR,261000,1,2008,11,PR
7,2008-11-WECC,655500,3,2008,11,WECC
8,2008-12-HI,294000,1,2008,12,HI
9,2008-12-NPCC,1389757,4,2008,12,NPCC


In [14]:
ng_fields = [ 'contract_type', 'fuel_cost', 'natural_gas_delivery_contract_type', 'natural_gas_supply_contract_type', 'natural_gas_transportation_service', 'operator_id',  'plant_id', 'plant_state', 'primary_transportation_mode', 'purchase_type', 'quantity', 'regulated', "reporting_frequency", "secondary_transportation_mode", "supplier", "combined_heat_and_power_plant", "nerc_region", "reported_prime_mover", "mmbtu_per_unit_gen", "year_month"]

df_ng = df[df["fuel_group"] == "Natural Gas"][ng_fields]

In [15]:
df_ng.head()

,contract_type,fuel_cost,natural_gas_delivery_contract_type,natural_gas_supply_contract_type,natural_gas_transportation_service,operator_id,plant_id,plant_state,primary_transportation_mode,purchase_type,quantity,regulated,reporting_frequency,secondary_transportation_mode,supplier,combined_heat_and_power_plant,nerc_region,reported_prime_mover,mmbtu_per_unit_gen,year_month
2,C,863.1,N/A,N/A,F,195,3,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,N,SERC,CA,1.04,2008-01
3,C,863.1,N/A,N/A,F,195,3,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,N,SERC,CT,1.04,2008-01
4,C,863.1,N/A,N/A,F,195,3,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,N,SERC,ST,1.04,2008-01
8,S,726.0,N/A,N/A,F,195,7,AL,N/A,N/A,2341,REG,A,N/A,VARIOUS,N,SERC,ST,1.03,2008-01
16,C,852.9,N/A,N/A,F,5701,9,TX,PL,N/A,23047,REG,A,N/A,ONE ENERGY MARKETING AND TRADING,N,WECC,GT,1.02,2008-01


In [121]:
df_nercs.to_sql("disturbance", conn)

/Users/lhinkesjones/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [16]:
df_ng["year_month_nerc"] = df_ng.apply(lambda x: x.year_month + "-" + x["nerc_region"], axis=1)

In [17]:
def temp_func(x):
    vals = x.split("-")
    vals[1] = vals[1].zfill(2)
    return "-".join(vals)
df_nercs.year_month_nerc = df_nercs.year_month_nerc.apply(temp_func)

In [18]:
 df_nercs[df_nercs["year_month_nerc"] == '2008-01-SERC']["Number of Customers Affected_x"].values[0]

110000

In [19]:
df_ng.head()

,contract_type,fuel_cost,natural_gas_delivery_contract_type,natural_gas_supply_contract_type,natural_gas_transportation_service,operator_id,plant_id,plant_state,primary_transportation_mode,purchase_type,quantity,regulated,reporting_frequency,secondary_transportation_mode,supplier,combined_heat_and_power_plant,nerc_region,reported_prime_mover,mmbtu_per_unit_gen,year_month,year_month_nerc
2,C,863.1,N/A,N/A,F,195,3,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,N,SERC,CA,1.04,2008-01,2008-01-SERC
3,C,863.1,N/A,N/A,F,195,3,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,N,SERC,CT,1.04,2008-01,2008-01-SERC
4,C,863.1,N/A,N/A,F,195,3,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,N,SERC,ST,1.04,2008-01,2008-01-SERC
8,S,726.0,N/A,N/A,F,195,7,AL,N/A,N/A,2341,REG,A,N/A,VARIOUS,N,SERC,ST,1.03,2008-01,2008-01-SERC
16,C,852.9,N/A,N/A,F,5701,9,TX,PL,N/A,23047,REG,A,N/A,ONE ENERGY MARKETING AND TRADING,N,WECC,GT,1.02,2008-01,2008-01-WECC


In [20]:
df["year_month_nerc"] = df.apply(lambda x: x.year_month + "-" + x["nerc_region"], axis=1)

In [ ]:
cursor.execute("""drop table combined5""", conn)

In [148]:
df.to_sql("combined5", conn)

In [21]:
df_temp = pd.DataFrame(cursor.execute("""
select * from combined5 c
Left join disturbance d
on d.year_month_nerc = c.year_month_nerc

""").fetchall())

In [22]:
df_temp.columns = [description[0] for description in cursor.description]


In [23]:
df_temp.head()

,index,average_ash_content,average_heat_content,average_mercury_content,average_sulfur_content,chlorine_content,coalmine_county,coalmine_msha_id,coalmine_name,coalmine_state,coalmine_type,contract_expiration_date,contract_type,energy_source,fuel_cost,fuel_group,moisture_content,month,natural_gas_delivery_contract_type,natural_gas_supply_contract_type,natural_gas_transportation_service,operator_id,operator_name,plant_id,plant_name,plant_state,primary_transportation_mode,purchase_type,quantity,regulated,reporting_frequency,secondary_transportation_mode,supplier,year,plant_id_gen,combined_heat_and_power_plant,nuclear_unit_id,plant_name_gen,operator_name_gen,operator_id_gen,state_gen,census_region,nerc_region,reserved_,naics_code,eia_sector_number,sector_name,reported_prime_mover,reported_fuel_type_code,aer_fuel_type_code,reserved_1,reserved_2,physical_unit_label,total_fuel_consumption_quantity,electric_fuel_consumption_quantity,total_fuel_consumption_mmbtu,elec_fuel_consumption_mmbtu,net_generation_megawatthours,year_gen,quantity_gen,elec_quantity_gen,mmbtu_per_unit_gen,tot_mmbtu_gen,elec_mmbtu_gen,netgen_gen,year_month,year_month_nerc,index,year_month_nerc,Number of Customers Affected_x,Number of Customers Affected_y,year,month,nerc
0,0,5.4,23.100,NaN,0.49,NaN,IMP,0,MINA PRIBBENOW,CL,SU,408.0,C,BIT,213.5,Coal,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,RV,N/A,259412,REG,M,N/A,INTEROCEAN COAL,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,ST,BIT,COL,None,None,short tons,4208251,4208251,94561606,94561606,9472859,2008,340778,340778,22.59,7697153,7697153,778699,2008-01,2008-01-SERC,NaN,None,NaN,NaN,None,None,None
1,1,5.7,22.800,NaN,0.48,NaN,IMP,0,MINA PRIBBENOW,CL,SU,408.0,C,BIT,211.5,Coal,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,RV,N/A,52241,REG,M,N/A,INTEROCEAN COAL,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,ST,BIT,COL,None,None,short tons,4208251,4208251,94561606,94561606,9472859,2008,340778,340778,22.59,7697153,7697153,778699,2008-01,2008-01-SERC,NaN,None,NaN,NaN,None,None,None
2,2,0.0,1.039,NaN,0.00,NaN,None,0,None,None,None,None,C,NG,863.1,Natural Gas,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,CA,NG,NG,None,None,mcf,3433,3433,3559,3559,1609033,2008,2400,2400,1.04,2489,2489,146327,2008-01,2008-01-SERC,NaN,None,NaN,NaN,None,None,None
3,3,0.0,1.039,NaN,0.00,NaN,None,0,None,None,None,None,C,NG,863.1,Natural Gas,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,CT,NG,NG,None,None,mcf,30932077,30932077,32090118,32090118,2858195,2008,2781102,2781102,1.04,2884003,2884003,244784,2008-01,2008-01-SERC,NaN,None,NaN,NaN,None,None,None
4,4,0.0,1.039,NaN,0.00,NaN,None,0,None,None,None,None,C,NG,863.1,Natural Gas,NaN,1,N/A,N/A,F,195,Alabama Power Co,3,Barry,AL,PL,N/A,2783619,REG,M,N/A,BAY GAS PIPELINE,2008,3,N,0,Barry,Alabama Power Co,195,AL,ESC,SERC,None,22,1,Electric Utility,ST,NG,NG,None,None,mcf,837652,837652,871270,871270,87265,2008,109281,109281,1.04,113543,113543,11487,2008-01,2008-01-SERC,NaN,None,NaN,NaN,None,None,None


In [24]:
df_temp["Number of Customers Affected_x"] = df_temp["Number of Customers Affected_x"].fillna(value=0)
df_temp["Number of Customers Affected_y"] = df_temp["Number of Customers Affected_y"].fillna(value=0)

In [25]:
df_temp = df_temp.rename(columns={"Number of Customers Affected_y": "number_affected", "Number of Customers Affected_x": "disturbance_events"})

In [ ]:
cursor.execute("""drop table combined5""", conn)
df.to_sql("combined5", conn)

In [28]:
ng_fields = [ 'contract_type', 'fuel_cost', 'natural_gas_delivery_contract_type', 'natural_gas_supply_contract_type', 'natural_gas_transportation_service', 'operator_id',  'plant_id', 'plant_state', 'primary_transportation_mode', 'purchase_type', 'quantity', 'regulated', "reporting_frequency", "secondary_transportation_mode", "supplier", "combined_heat_and_power_plant", "nerc_region", "reported_prime_mover", "mmbtu_per_unit_gen", "disturbance_events", "number_affected"]

df_ng = df_temp[df_temp["fuel_group"] == "Natural Gas"][ng_fields]
not_encode = []
to_encode = []
for fld, fld_type in df_ng[ng_fields].dtypes.items():
    if "object" == fld_type:
        to_encode.append(fld)
    else:
        not_encode.append(fld)
temp1 = df_ng[not_encode].reset_index()
temp2 = pd.get_dummies(df_ng[to_encode]).reset_index()
df_ng = temp1.merge(temp2, on="index").set_index("index")

In [29]:
X = df_ng.drop(columns="fuel_cost")
y = (df_ng.fuel_cost > df_ng.fuel_cost.quantile(.9))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [30]:
model = DecisionTreeClassifier(random_state=23, max_depth=10)
results = model.fit(X_train, y_train)


In [180]:
results.score(X_test, y_test)

0.912666927418003

In [33]:
cross_val_score(results, X, y, cv=5)

array([0.54463504, 0.89961074, 0.90724702, 0.9066324 , 0.90318495])

In [34]:
for a,b in sorted(zip(X.columns, results.feature_importances_), key=lambda x: x[1], reverse=True):
    print(a,b)

primary_transportation_mode_PL 0.195822865142861
quantity 0.11155771835921047
number_affected 0.04573004388983382
operator_id_19876 0.04485184120557328
purchase_type_N/A 0.04095981127209089
operator_id_16534 0.04033535944560916
mmbtu_per_unit_gen 0.0385514475601007
nerc_region_SPP 0.03651207661000651
plant_id_56328 0.02669052158696583
plant_state_NY 0.023114795011318804
supplier_VARIOUS 0.019560685442648145
nerc_region_FRCC 0.01799693950869678
supplier_MUNIGAS - BP ENERGY 0.016920916335586237
supplier_LG&E 0.016722264376815305
operator_id_11249 0.01648261743120721
contract_type_C 0.015328111427231015
plant_state_WA 0.015100062607356763
plant_id_141 0.014941063749267522
supplier_BPE 0.013578752386242586
supplier_POWEREX 0.01198205784804323
operator_id_12745 0.011877066332300294
plant_id_2790 0.011876446845635706
supplier_constellation 0.010641444829108974
plant_state_NC 0.00990200605572105
operator_id_15474 0.00989242477041212
plant_id_1730 0.009038895579338013
supplier_BEAR ENERGY 0.00

plant_id_3295 0.0
plant_id_3298 0.0
plant_id_3325 0.0
plant_id_3344 0.0
plant_id_3393 0.0
plant_id_3403 0.0
plant_id_3405 0.0
plant_id_3406 0.0
plant_id_3456 0.0
plant_id_3457 0.0
plant_id_3459 0.0
plant_id_3476 0.0
plant_id_3478 0.0
plant_id_3482 0.0
plant_id_3484 0.0
plant_id_3485 0.0
plant_id_3548 0.0
plant_id_3559 0.0
plant_id_3576 0.0
plant_id_358 0.0
plant_id_3601 0.0
plant_id_3602 0.0
plant_id_3604 0.0
plant_id_3609 0.0
plant_id_3611 0.0
plant_id_3612 0.0
plant_id_3613 0.0
plant_id_3627 0.0
plant_id_3628 0.0
plant_id_3630 0.0
plant_id_3631 0.0
plant_id_3648 0.0
plant_id_377 0.0
plant_id_3775 0.0
plant_id_3796 0.0
plant_id_3797 0.0
plant_id_389 0.0
plant_id_3944 0.0
plant_id_3946 0.0
plant_id_3982 0.0
plant_id_399 0.0
plant_id_3991 0.0
plant_id_3992 0.0
plant_id_400 0.0
plant_id_4005 0.0
plant_id_4014 0.0
plant_id_404 0.0
plant_id_4040 0.0
plant_id_4041 0.0
plant_id_4042 0.0
plant_id_4048 0.0
plant_id_4057 0.0
plant_id_4072 0.0
plant_id_4076 0.0
plant_id_4078 0.0
plant_id_408 0.0

supplier_KOCH ENERGY SERVICE (KES) 0.0
supplier_KOCH TRADING AND SUPPLIES 0.0
supplier_Kinder Morgan Texas Pipeline 0.0
supplier_LACLED 0.0
supplier_LACLEDE 0.0
supplier_LACLEDE ENERGY RESOURCES 0.0
supplier_LAKESHORE ENERGY SERVICES,LLC 0.0
supplier_LCRA GAS 0.0
supplier_LIG 0.0
supplier_LOUIS DREYFUS ENERGY 0.0
supplier_LOUISIANA INTERSTATE GAS 0.0
supplier_Ldreyfus 0.0
supplier_MACQUARIE ENERGY 0.0
supplier_MADISON GAS AND ELECTRIC CO 0.0
supplier_MAIN STREET NATURAL GAS,Inc 0.0
supplier_MANSFIELD 0.0
supplier_MARATHON OIL COMPANY 0.0
supplier_MARATHON PETROLEUM 0.0
supplier_MARITIMES & NORTHEAST 0.0
supplier_MARK WEST 0.0
supplier_MARTIN ENERGY SERVICES 0.0
supplier_MASEFIELD 0.0
supplier_MCE 0.0
supplier_MCQUARIE COOK ENERGY 0.0
supplier_MEMS,INC. 0.0
supplier_MERCURIA 0.0
supplier_MERCURIA ENERGY TRADING INC 0.0
supplier_MERRILL LYNCH 0.0
supplier_MERRILL LYNCH COMMODITIES, INC. 0.0
supplier_MERRIMAC ENERGY GROUP 0.0
supplier_METROPOLITAN UTILITIES DISTRICT 0.0
supplier_MGC 0.0
s